In [1]:
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dense, Dropout, Input, Flatten, Conv1D, MaxPooling1D, Embedding, Dropout, LSTM
from keras.models import Sequential, Model
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping


import numpy as np

import pandas as pd


In [2]:
# carregar modelo word2vec
word2vec = KeyedVectors.load_word2vec_format('../skip_s300.txt')
vocab_size, embedding_size = word2vec.vectors.shape

In [3]:
# carregar dados de treinamento do CSV
df = pd.read_csv('../train.csv', header=None, names=['texto', 'classe'])
X = df['texto'].values
y = df['classe'].values

In [4]:
# tokenizar o texto
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# padronizar o tamanho das sequencias
max_length = max([len(text) for text in X])
X = pad_sequences(X, maxlen=max_length, padding='post')

In [5]:
embedding_layer = Embedding(vocab_size, embedding_size, weights=[word2vec.vectors], input_length=max_length, trainable=False)
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
conv_layer1 = Conv1D(filters=32, kernel_size=3, activation='relu')(embedded_sequences)
conv_layer1 = MaxPooling1D(pool_size=2)(conv_layer1)
conv_layer2 = Conv1D(filters=64, kernel_size=3, activation='relu')(conv_layer1)
conv_layer2 = MaxPooling1D(pool_size=2)(conv_layer2)
lstm_layer = LSTM(64)(conv_layer2)
output_layer = Dense(7, activation='softmax')(lstm_layer)

model = Model(sequence_input, output_layer)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model.summary()


Simplified convolutional neural network
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 18)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 18, 300)           278881800 
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 32)            28832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 8, 32)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 6, 64)             6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 64)             0         
_________________________________________________________________
lstm (LSTM)          

In [6]:
# separar dados em treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, 7)
y_val = to_categorical(y_val, 7)

In [7]:
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

In [8]:
# treinar modelo
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping_monitor])

Epoch 1/30
26/26 [==============================] - 3s 29ms/step - loss: 1.6336 - acc: 0.4520 - val_loss: 1.0546 - val_acc: 0.6995
Epoch 2/30
26/26 [==============================] - 0s 7ms/step - loss: 0.6267 - acc: 0.8436 - val_loss: 0.3300 - val_acc: 0.9507
Epoch 3/30
26/26 [==============================] - 0s 5ms/step - loss: 0.2086 - acc: 0.9594 - val_loss: 0.2948 - val_acc: 0.8966
Epoch 4/30
26/26 [==============================] - 0s 4ms/step - loss: 0.1098 - acc: 0.9791 - val_loss: 0.1013 - val_acc: 0.9803
Epoch 5/30
26/26 [==============================] - 0s 4ms/step - loss: 0.0590 - acc: 0.9901 - val_loss: 0.0930 - val_acc: 0.9606
Epoch 6/30
26/26 [==============================] - 0s 5ms/step - loss: 0.0523 - acc: 0.9889 - val_loss: 0.1040 - val_acc: 0.9704
Epoch 7/30
26/26 [==============================] - 0s 4ms/step - loss: 0.0363 - acc: 0.9889 - val_loss: 0.1476 - val_acc: 0.9557
Epoch 8/30
26/26 [==============================] - 0s 5ms/step - loss: 0.0227 - acc: 0.9